# Project: LSTM_module (predict)

학습된 모델로 음성파일 스팸 예측

## 순서

1. import: 필요한 모듈 import
2. 파일 불러오기: 모델 학습에 사용된 객체, 변수 등 불러오기
3. STT: 음성파일 STT변환
4. 전처리: 텍스트 토큰화
5. 예측: 스팸 탐지

## 모듈 import

In [64]:
import sys
import json
import pickle
import re
import numpy as np

import speech_recognition as sr

import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

## GPU 셋팅

GPU 사용 시 활성화

In [65]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## 파일 이름 선언

필요 변수, 모델, 토큰 객체 등 파일 이름 선언

In [66]:
file_name = "LSTM_module_ver4.0"

토큰화 객체 불러오기

In [67]:
with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

변수 불러오기

In [68]:
with open(file_name+'_variable.json') as f:
    var = json.load(f)

불용어 단어 불러오기

In [69]:
with open(file_name+'_stopwords.json') as f:
    stopwords = json.load(f)

불러온 변수 초기화

In [70]:
min_data = var['min_data'] # 데이터 문자열 최소 길이
data_length = var['data_length'] # 데이터 문자열 최대 길이
split_list = var['split_list'] # 문장 단위로 분리하기 위한 글자 리스트
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)
trunc_type = var['trunc_type']
padding_type = var['padding_type']

print('min_data:',min_data)
print('data_length:',data_length)
print('max_len:',max_len)
print('trunc_type:',trunc_type)
print('padding_type:',padding_type)

min_data: 10
data_length: 100
max_len: 100
trunc_type: post
padding_type: post


## STT 변환

음성파일 STT변환
* `language`: 언어 설정
* 약 80~100MB 용량 제한

In [71]:
audio_file = ["spam1.wav", "spam2.wav", "spam1.flac", "spam2.flac", "ham1.wav", "ham2.wav", "ham1.flac", "ham2.flac"]

stt_list = []
for a in audio_file:
    r = sr.Recognizer()
    harvard = sr.AudioFile(a) # 100MB 용량 제한
    with harvard as source:
        audio = r.record(source)

    try:
        t = r.recognize_google(audio, language='ko-KR')
    except:
        t = '녹음 내용이 존재하지 않습니다.'
        
    stt_list.append(t)

## 텍스트 데이터 분리

**findIndex**: 문장이 끝나는 부분의 index값을 찾기위한 함수

In [72]:
def findIndex(data_list, split_list, start_index):
    index_list = []
    for i in split_list:
        index = data_list.find(i, start_index)
        index_list.append(index)
        
    index_list = [i for i in index_list if i not in [-1]]
    if index_list == []:
        return -1
    
    index = min(index_list)
    
    return index     

`data_length` 값 이후에 `split_list` 에 있는 문자가 나오면 자르기

In [73]:
result = []
for t in stt_list:
    t = re.sub("네|[^가-힣 ]", "", t) # 특수문자 제거
    index = findIndex(t, split_list, data_length)

    i = 0
    t_list = []
    while index != -1:
        x = t[i:index+2]
        t_list.append(x)

        i = index+2
        index = findIndex(t, split_list, i+data_length)
    else:
        x = t[i:]
        if len(x) > min_data:
            t_list.append(x) # 텍스트 마지막 부분 추가
            
    result.append(t_list)

## 전처리: 토큰화 및 불용어 처리

Okt 객체 선언

In [74]:
okt = Okt()

**preTreatment**: 토큰화 및 불용어 처리 함수

In [75]:
def preTreatment(data):
    global stopwords
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp

**sentiment_predict**: 정수 인코딩 후 모델 예측 함수

In [76]:
def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(data, maxlen = max_len, truncating = trunc_type, padding = padding_type)
    score = float(model.predict(pad_new))
#     print(score)
#     print('----------------------')
    return score

## 모델 예측

학습된 모델 불러오기

In [77]:
model = tf.keras.models.load_model(file_name+'.h5', compile = False)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         657100    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 128)         117248    
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,037,645
Trainable params: 1,037,645
Non-trainable params: 0
_________________________________________________________________


분리한 각 데이터 확률 예측 후 평균 구하기

In [78]:
for t_list in result:
    score_sum = 0.0
    
    for txt in t_list:
        print(txt)
        data = preTreatment(txt)
        score_sum += sentiment_predict(data)

    score_result = score_sum / len(t_list)
    
    print("{:.2f}% 확률로 스팸입니다.\n".format(score_result * 100))

여보세요 여보세요 여기 서울중앙지검 잠시 통화 괜찮으십니까 다름이 아니라 본인 명의도용사건 관련해서 몇 가지 사실 확인 차 연락 드렸어요 저는 서울중앙지검 특별수사팀 제일 보강은 영숙아이라고 합니다 
예 부모님께서 대구 달서구 출신인    감동이라는 사람 아는 사람이십니까 최고야 최관동이라고 합니다 최관동 처음 들어 보는데 그 대구 달서구 출신이구요   되고 저기 그 농협은행 직원이었습니다 
전혀 모르는 사람 맞으신가요 예 알겠습니다 제가 그 사람에 대해서 외제차 여쭤봐야 만요 저희 수사팀에서 이번에 대구 달서구에서 발생한 금융사기를 조사하던중 안동을 중심으로 일정 대로 검거했습니다 
하다 보니까 신분증 신용카드도 했었는데요 농협 은행 통장 발급 돼서 연락을 드린 거고요 혹시 통장에 대해서 알고 있는 내용 있습니까 아니요 전혀 없는데요 어떻게 해야 되나요 알겠습니다 
대해서 저희 측에 더 확인이 잘 부분이고요 년 월 일 목요일 있고요 달서구 대덕 지점에서 발급 된 걸로 확인이 되거든요 어머님께서 발급받은 통장들이 왔습니까 어디서 농협은행과 신한은행 같은 경우 통장 별들이 년 월 일 목요일 이고요 
대구 달서구 대덕 지점에서 발급이 된 걸로 확인이 되거든요 아니요 거기서 저녁 대구 달서구 따로 연구실에 없으신 거 맞으시고요 알겠습니다 화장실 방법은에 대해서 제가 지금 현재 연락을 드렸는데요 
친구 단말기에  연락을 드린 거고요 내선번호로 연락드리면 발급시스템 폰으로 오셔서 안 받는 경우가 많더라고요 그렇기 때문에 해당 부분을 저희가 녹취는 단말기로 연락을 드린 거고요 분위기에서 좀 양해를 해 주셔야 하는데 일단 조사를 하고 있는데요 
현 시점에서 사건 현장과 용이점 묵호에서 본인명의 천상두 저 발견되었기 때문에 저희 쪽에서는 본인이 통장을 판매할 수 있는지 아니면 개인정보 유출로 인한 피해 사실을 있는지 그 부분에 대해서 조사를 하고 있는 중이고요 
무엇보다도 중요한 점이 통장이 사용됨으로써 다수의 피해자와의 성장하는 피해 금요일까지 발생을 했기 때문에 되지 않

## 예측 확률 출력

In [79]:
# print("{:.2f}% 확률로 스팸입니다.\n".format(score_result * 100))